In [1]:
from kafka import KafkaProducer
from time import sleep
from json import dumps
import sys
# sys.path.append('D:/Projects/Crypto/Scripts/API')
sys.path.extend([
    'D:/Projects/Crypto/Scripts/API'
])
import CoinsInfo as ce

In [2]:
data = ce.coin_info(0)
# data['name']

{'name': 'Bitcoin', 'symbol': '₿', 'rank': 1, 'age': 5641, 'color': '#fa9e32', 'png32': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/32/btc.png', 'png64': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/64/btc.png', 'webp32': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/32/btc.webp', 'webp64': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/64/btc.webp', 'exchanges': 194, 'markets': 2969, 'pairs': 880, 'categories': [], 'allTimeHighUSD': 73781.24185982272, 'circulatingSupply': 19710859, 'totalSupply': 19710859, 'maxSupply': 21000000, 'links': {'website': 'https://bitcoin.org', 'whitepaper': 'https://bitcoin.org/bitcoin.pdf', 'twitter': None, 'reddit': 'https://reddit.com/r/bitcoin', 'telegram': None, 'discord': None, 'medium': None, 'instagram': None, 'tiktok': None, 'youtube': None, 'linkedin': None, 'twitch': None, 'spotify': None, 'naver': None, 'wechat': None, 'soundcloud': None}, 'rate': 65650.787027

In [3]:
while True:
    sleep(20)
    data = ce.coin_info(0)
    producer = KafkaProducer(bootstrap_servers=['52.14.28.124:9092'],
                            value_serializer=lambda x: dumps(x).encode('utf-8'))

    producer.send('crypto', value=data)

In [3]:
#Creating custom Exception
class APICallError(Exception):
    """Raised when there is an error during an API call"""
    pass

while True:
    sleep(20)
    try:
        
        # Fetch data from API
        data = ce.coin_info(0)
        if 'error' in data:
            raise APICallError(f"API Error: {data['error']['description']}")

    except APICallError as api_err:
        print(f"API call failed: {api_err}")

    else:
        producer = KafkaProducer(bootstrap_servers=['52.14.28.124:9092'],
                                value_serializer=lambda x: dumps(x).encode('utf-8'))
        print(producer.send('crypto', value=data))

{'name': 'Bitcoin', 'symbol': '₿', 'rank': 1, 'age': 5641, 'color': '#fa9e32', 'png32': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/32/btc.png', 'png64': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/64/btc.png', 'webp32': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/32/btc.webp', 'webp64': 'https://lcw.nyc3.cdn.digitaloceanspaces.com/production/currencies/64/btc.webp', 'exchanges': 194, 'markets': 2969, 'pairs': 880, 'categories': [], 'allTimeHighUSD': 73781.24185982272, 'circulatingSupply': 19710859, 'totalSupply': 19710859, 'maxSupply': 21000000, 'links': {'website': 'https://bitcoin.org', 'whitepaper': 'https://bitcoin.org/bitcoin.pdf', 'twitter': None, 'reddit': 'https://reddit.com/r/bitcoin', 'telegram': None, 'discord': None, 'medium': None, 'instagram': None, 'tiktok': None, 'youtube': None, 'linkedin': None, 'twitch': None, 'spotify': None, 'naver': None, 'wechat': None, 'soundcloud': None}, 'rate': 65649.409674